Hough-Transformation

2. Implementieren Sie die Hough-Transformation f ̈ur Geraden. Verwenden Sie die Hesse’sche
Normalform (HNF) und orientieren Sie sich bez ̈uglich des Programmieransatzes an die
Vorlesungsunterlagen (bzw. Buch Seite 162). Wenden Sie die Hough-Transformation
auf das Bild noisy-lines an.
•Prototyp: houghArray = linearHT(im edge, angle steps, angle steps)
houghArray: Ergebnisbild nach Hough-Transformation (int)
im edge: Eingangsbild in Bin ̈arform (0,1)
angle stepsize: Schrittweite Winkel (int)
radius stepsize: Schrittweite Radius (int)
Beispielaufruf: linearHT(image, 100, 100);

In [14]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import io
from skimage.feature import canny
import os
import pandas as pd

# Bild einlesen
#x{i}.jpg
images = []
for i in range (1, 9):
    image_path = f'../Ordner des ungeordneten/x{i}.png'
    if os.path.exists(image_path):
        images.append(io.imread(image_path, as_gray=True))
    image_path = f'../Ordner des ungeordneten/o{i}.png'
    if os.path.exists(image_path):
        images.append(io.imread(image_path, as_gray=True))
    image_path = f'../Ordner des ungeordneten/nothing{i}.png'
    if os.path.exists(image_path):
        images.append(io.imread(image_path, as_gray=True))
print(f'Found {len(images)} images.')

Found 18 images.


In [15]:
def linearHT(im_edge, angle_steps, radius_steps):
    height, width = im_edge.shape
    x_ctr, y_ctr = width // 2, height // 2
    
    # Schrittweiten berechnen
    d_ang = np.pi / angle_steps
    r_max = np.sqrt(x_ctr**2 + y_ctr**2)
    d_rad = (2 * r_max) / radius_steps
    
    # Hough-Akkumulator initialisieren
    hough_array = np.zeros((angle_steps, radius_steps), dtype=int)
    
    # Hough-Akkumulator füllen
    y_idxs, x_idxs = np.where(im_edge > 0)
    
    for i in range(len(y_idxs)):
        y, x = y_idxs[i], x_idxs[i]
        x_shifted = x - x_ctr
        y_shifted = y - y_ctr
        
        for a in range(angle_steps):
            theta = d_ang * a
            r = int(np.round((x_shifted * np.cos(theta) + y_shifted * np.sin(theta)) / d_rad) + radius_steps / 2)
            if 0 <= r < radius_steps:
                hough_array[a, r] += 1
    
    return hough_array


In [16]:
def linearHT_with_threshold(im_edge, angle_steps, radius_steps, threshold_ratio=0.7):

    # Standard Hough-Transformation ausführen
    hough_array = linearHT(im_edge, angle_steps, radius_steps)
    
    # Schwelloperation durchführen
    max_value = np.max(hough_array)
    threshold = int(max_value * threshold_ratio)
    hough_thresholded = np.zeros_like(hough_array)
    hough_thresholded[hough_array >= threshold] = 1
    
    # Peaks identifizieren (Winkel und Radius der Linien)
    peak_indices = np.where(hough_array >= threshold)
    angles = peak_indices[0] * (np.pi / angle_steps)
    distances = peak_indices[1] - radius_steps // 2
    peak_params = list(zip(angles, distances))
    
    return hough_array, hough_thresholded, peak_params


In [17]:
# def extract_hough_features(hough_array):

#     max_val = np.max(hough_array)
#     mean_val = np.mean(hough_array)
#     std_val = np.std(hough_array)
#     strong_lines = np.sum(hough_array > 20)  # Zähle starke Peaks
    
#     return {
#         'max': max_val,
#         'mean': mean_val,
#         'std': std_val,
#         'strong_lines': strong_lines
#     }


In [18]:
max_hough = [[], [], []]
anz_inizes = [[], [], []]
indices_list = [[], [], []]
anz_lines = [[], [], []]

for i, image in enumerate(images):
    edges = canny(image, sigma=1.0, low_threshold=0.1, high_threshold=0.2)
    # Hough-Transformation anwenden
    hough_array = linearHT(edges, 180, 180)

    # Hough-Raum anzeigen
    # plt.figure(figsize=(7, 5))
    # plt.imshow(hough_array, cmap='hot', aspect='auto', 
    #         extent=[0, 180, -90, 90])
    # plt.title("Hough-Transformation")
    # plt.xlabel("Winkel (Grad)")
    # plt.ylabel("Abstand")
    # plt.colorbar(label='Akkumulator-Werte')
    # plt.tight_layout()
    # plt.figure()
    # plt.imshow(image, cmap='gray')  # cmap='gray' für korrekte Graustufenanzeige
    # plt.title("Noisy Lines (Originalbild)")
    # plt.axis('off')
    
    
    
    # Linienparameter aus den Peaks im Hough-Raum finden
    threshold = 0.7 * np.max(hough_array)  # 70% des Maximalwertes als Schwellwert
    
    #print(f"Maximalwert im Hough-Raum: {np.max(hough_array)}")
    indices = np.where(hough_array >= threshold)
    
    #print(f"Anzahl der Indizes über dem Schwellwert: {len(indices[0])}")
    #print(f"Indices: {indices}")

    # Konvertiere Indizes zurück zu Winkel und Abstand
    #angles = indices[0] * (np.pi / 180)
    #distances = indices[1] - 180//2
    #print(f"angels= {angles}, distances={distances}")
    
    _, hough_thresholded, peak_params = linearHT_with_threshold(edges, 180, 180, 0.3)
    lines = len(peak_params)
    #print(f"Gefundene Linien: {lines}")

    
    # Gefundene Linien zeichnen
    # height, width = image.shape
    # for angle, dist in peak_params:
    #     # Konvertiere von Hessescher Normalform zu Endpunktform
    #     a = np.cos(angle)
    #     b = np.sin(angle)
    #     x0 = a * dist + width // 2
    #     y0 = b * dist + height // 2
        
    #     # Berechne Endpunkte der Liniensegmente
    #     x1 = int(x0 + 1000 * (-b))
    #     y1 = int(y0 + 1000 * (a))
    #     x2 = int(x0 - 1000 * (-b))
    #     y2 = int(y0 - 1000 * (a))
        
    #     # Zeichne die Linie
    #     plt.plot([x1, x2], [y1, y2], 'b-', linewidth=1)
    # plt.text(10, 20, f"Gefundene Linien: {len(peak_params)}", 
    #         color='white', fontsize=12, backgroundcolor='black')
    # plt.show()
    # features = extract_hough_features(hough_array)
    # print(f"Bild {i+1}: Max: {features['max']}, Mean: {features['mean']}, Std: {features['std']}, Starke Linien: {features['strong_lines']}")
    
    if lines > 30000:
        max_hough[2].append(np.max(hough_array))
        anz_inizes[2].append(len(indices[0]))
        anz_lines[2].append(lines)
        print("Empty")
    elif lines > 500:
        max_hough[1].append(np.max(hough_array))
        anz_inizes[1].append(len(indices[0]))
        anz_lines[1].append(lines)
        print("O")
    else:
        max_hough[0].append(np.max(hough_array))
        anz_inizes[0].append(len(indices[0]))
        anz_lines[0].append(lines)
        print("X")
    
    
    
#print(f"Max_hough: {max_hough}")
#print(f"Anzahl Indizes über dem 70% schwellwert: {anz_inizes}")
#print(f"ANzahl der gefunden Linien{anz_lines}")

data = {
    'X':     [max_hough[0], anz_inizes[0], anz_lines[0]],
    'O':     [max_hough[1], anz_inizes[1], anz_lines[1]],
    'Empty': [max_hough[2], anz_inizes[2], anz_lines[2]]
}

# Erzeuge DataFrame mit passenden Zeilenbezeichnungen
df = pd.DataFrame(data, index=["max_hough", "anz_indizes", "anz_lines"])

# Ausgabe der Tabelle
print(df)




X
O
Empty
X
O
Empty
X
O
X
O
X
O
X
O
X
O
X
O
                                                   X  \
max_hough        [45, 46, 48, 34, 43, 378, 419, 246]   
anz_indizes             [13, 9, 15, 16, 6, 9, 3, 24]   
anz_lines    [300, 401, 296, 473, 210, 183, 94, 287]   

                                                          O           Empty  
max_hough               [23, 22, 43, 27, 30, 204, 124, 111]          [0, 0]  
anz_indizes              [28, 142, 1, 86, 27, 107, 76, 112]  [32400, 32400]  
anz_lines    [4433, 5520, 521, 4072, 791, 1442, 3191, 2807]  [32400, 32400]  
